# Open a restaurant in Toronto

__Opening the restaurant in the right place may bring additional profit to the business.__

Using Foursquare data, I will get the information on Paris neighborhoods.

Then, I will explore the top venues of each neighboorhood. Based on the result, I will highlight the neighboorhoods that have the following features:

1) high public flow (a lot of btheaters, gyms, shops, parks)

2) low number of already functioning restaurants

Based on the result, I will set the down threshold for number of those venues and upper threshold on number of already open restaurants

The data will be clustered on two types (neighboorhoods that fit the restaurant opening, and the other that do not)

----------------------------------------------------------------------------------------------

import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

 # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup # this module helps in web scrapping.

Retrieve the data from the wikipidea page

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text

Create a BeautifulSoup object

In [3]:
soup=BeautifulSoup(data,'html5lib')

Following the hints, assign the corresponding cells to a list of dictionaries. The list of dictionaries is converted to dataframe

In [148]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head(3)
df.shape

(103, 3)

Import the geospatial coordiantes of neighborhoods

In [5]:
latlng=pd.read_csv('Geospatial_Coordinates.csv')

Add the latitude and longitude data to the dataframe with neigborhood data

In [157]:
latitude=[]
longitude=[]
for postal_code in df['PostalCode']:
    latitude.append(latlng[latlng['Postal Code']==postal_code]['Latitude'])
    longitude.append(latlng[latlng['Postal Code']==postal_code]['Longitude'])
df[['Latitude']]=latitude
df[['Longitude']]=longitude
df.head(3)

,PostalCode,Borough,Neighborhood,Cluster Labels,Color Labels,Labels,Latitude,Longitude
0,M3A,North York,Parkwoods,0,0,Should not make a restaurant,43.753259,-79.329656
1,M4A,North York,Victoria Village,0,0,Should not make a restaurant,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,Should not make a restaurant,43.654260,-79.360636


In [7]:
print('The dataframe has {} neighborhoods.'.format(len(df['Neighborhood'].unique())))

The dataframe has 103 neighborhoods.


Locate Toronto coordinates

In [167]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Map of neighborhoods in Toronto

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)
# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


The Foursquare API will be called to gather the information on venues

My client ID is introduced:

In [10]:
CLIENT_ID = 'NBUHQJPXT0LANYUJ5QTSTHCDFWSWQ4HNXXEQ3IKWTLNGN2AE' # your Foursquare ID
CLIENT_SECRET = 'TZVYGVWFYUE4FDB4VWX3N0TR4SMMGZEIATELMOKASHC2RW2O' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50 # A default Foursquare API limit value

Get nearby venues

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
toronto_venues=getNearbyVenues(names=df['Neighborhood'],
                                 latitudes=df['Latitude'],
                                longitudes=df['Longitude']
                                )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

See the venues in each neighborhood

In [13]:
print(toronto_venues.shape)
toronto_venues.head()

(1007, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


See how many parks are in the city

In [14]:
len(toronto_venues[toronto_venues['Venue Category']=='Park'])

22

In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Health & Beauty Service,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Find all restaurant categories

In [16]:
restaurant_categories=[]
for restaurant_type in toronto_onehot.columns:
    if 'Restaurant' in restaurant_type:
        restaurant_categories.append(restaurant_type)

Determine number of restaurants per neighborhood

In [17]:
toronto_grouped_restaurants = toronto_onehot[restaurant_categories+['Neighborhood']].groupby('Neighborhood').sum()
toronto_grouped_restaurants["sum restaurants"] = toronto_grouped_restaurants.sum(axis=1)
toronto_grouped_restaurants=toronto_grouped_restaurants.reset_index()[['Neighborhood','sum restaurants']]
toronto_grouped_restaurants.head()

,Neighborhood,sum restaurants
0,Agincourt,1
1,"Alderwood, Long Branch",0
2,"Bathurst Manor, Wilson Heights, Downsview North",2
3,"Bedford Park, Lawrence Manor East",7
4,Berczy Park,1


Introduce a function to groupvenue categories containing a certain keyword

In [18]:
def grouped_df(toronto_onehot,keyword):
    venue_categories=[]
    for venue_type in toronto_onehot.columns:
        if keyword in venue_type:
                venue_categories.append(venue_type)
    venue_grouped = toronto_onehot[venue_categories+['Neighborhood']].groupby('Neighborhood').sum()
    venue_grouped["sum"] = venue_grouped.sum(axis=1)
    venue_grouped=venue_grouped.reset_index()[['Neighborhood','sum']]
    return  venue_grouped

Get the number of exhisting restaurant in the neighborhood

In [100]:
restaurant_grouped=grouped_df(toronto_onehot,'Restaurant')
restaurant_grouped=restaurant_grouped.set_index('Neighborhood')
restaurant_grouped.head()
#restaurant_grouped.loc['Agincourt', 'sum']

,sum
Neighborhood,
Agincourt,1
"Alderwood, Long Branch",0
"Bathurst Manor, Wilson Heights, Downsview North",2
"Bedford Park, Lawrence Manor East",7
Berczy Park,1


Get the number of gyms per neighborhood

In [68]:
gym_grouped=grouped_df(toronto_onehot,'Gym')

Get the number of parks per neighborhood

In [69]:
park_grouped=grouped_df(toronto_onehot,'Park')

Get the number of shops per neighborhood

In [70]:
shop_grouped=grouped_df(toronto_onehot,'Shop')

Get the number of theaters per neighborhood

In [71]:
theater_grouped=grouped_df(toronto_onehot,'Theater')

I want to compute points to highlight the advantages of each neighborhood, and then find the best neighborhoods with less than 3 restaurants in the neighborhood

In [72]:
park_grouped['park_points']=park_grouped[['sum']]*2
park_grouped.drop(columns=['sum'],inplace=True)

In [73]:
theater_grouped['theater_points']=theater_grouped[['sum']]*2
theater_grouped.drop(columns=['sum'],inplace=True)

In [74]:
gym_grouped['gym_points']=gym_grouped[['sum']]
gym_grouped.drop(columns=['sum'],inplace=True)

In [75]:
shop_grouped['shop_points']=shop_grouped[['sum']]
shop_grouped.drop(columns=['sum'],inplace=True)

In [78]:
gym_grouped.set_index('Neighborhood', inplace=True)
shop_grouped.set_index('Neighborhood', inplace=True)

Find the points for the neighborhood

In [89]:
venues_grouped=pd.concat([park_grouped, theater_grouped,gym_grouped, shop_grouped], axis=1)
venues_grouped['sum points']=venues_grouped.sum(axis=1)
venues_grouped=venues_grouped.reset_index()
venues_grouped.head()

,Neighborhood,park_points,theater_points,gym_points,shop_points,sum points
0,Agincourt,0,0,0,0,0
1,"Alderwood, Long Branch",0,0,1,1,2
2,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,4,4
3,"Bedford Park, Lawrence Manor East",0,0,0,2,2
4,Berczy Park,0,0,0,0,0


Show the best neigborhoods in terms of points

In [93]:
venues_grouped=venues_grouped.sort_values(by=['sum points'],ascending=False)
venues_grouped.head()

,Neighborhood,park_points,theater_points,gym_points,shop_points,sum points
36,"Harbourfront East, Union Station, Toronto Islands",2,0,1,13,16
73,"Toronto Dominion Centre, Design Exchange",0,0,4,10,14
33,"Garden District, Ryerson",0,4,0,9,13
12,Church and Wellesley,2,0,3,8,13
15,"Commerce Court, Victoria Hotel",0,0,3,9,12


Exclude neighborhoods with high number of restaurants

In [158]:
num_neighborhoods=5
good_neighborhoods=[]
for neighborhood in venues_grouped['Neighborhood']:
    
    if restaurant_grouped.loc[neighborhood, 'sum']<3:
        good_neighborhoods.append(neighborhood)
    if len(good_neighborhoods)==num_neighborhoods:
        break


Make color labels for neighborhoods where it's worth to make a restaurant, and where it's not

In [159]:
Color_Labels=[]
#Labels=[]
for neighborhood in df['Neighborhood']:
    if neighborhood in good_neighborhoods:
        Color_Labels.append(1)
        #Labels.append('Worth to make a restaurant')
    else:
        Color_Labels.append(0)
        #Labels.append('Should not make a restaurant')

Finally, we generate the datafrome for creating a folium map

In [161]:
good_neighborhoods_df=df
good_neighborhoods_df['Color Labels']=Color_Labels
good_neighborhoods_df['Labels']=Labels
good_neighborhoods_df.head()

,PostalCode,Borough,Neighborhood,Cluster Labels,Color Labels,Labels,Latitude,Longitude
0,M3A,North York,Parkwoods,0,0,Should not make a restaurant,43.753259,-79.329656
1,M4A,North York,Victoria Village,0,0,Should not make a restaurant,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,Should not make a restaurant,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0,0,Should not make a restaurant,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,0,0,Should not make a restaurant,43.662301,-79.389494


Create map

In [170]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
#toronto_merged=toronto_merged.dropna(subset=['Cluster Labels'])
df=df.astype({'Cluster Labels':'int32'})
# set color scheme for 2 colors
x = np.arange(2)
ys = [i + x + (i*x)**2 for i in range(2)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster,label in zip(df['Latitude'], df['Longitude'], df['Neighborhood'],df['Cluster Labels'],df['Labels']):
    label = folium.Popup(str(poi) + ' - '+ label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[color1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters